### installing the requirements
!pip install -r ../mlops-zoomcamp/02-experiment-tracking/requirements.txt
#### to run in cli:
#### mlflow ui --backend-store-uri sqlite:///mlflow.dbmlflow --backend-store-uri sqlite:///mlflow.db